In [1]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.quantization
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets,transforms


In [2]:
#Transformation to normalize the Data
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL images to tensors
    transforms.Normalize((0.1307,), (0.3081,))  # Normalize with mean and std dev for MNIST
])
mnist_train = datasets.MNIST(root='./data', 
                             train=True, 
                             transform=transform, 
                             download=True)
mnist_test = datasets.MNIST(root='./data', 
                            train=False, 
                            transform=transform, 
                            download=True)

X_train = mnist_train.data
y_train = mnist_train.targets
X_test = mnist_test.data
y_test = mnist_test.targets

X_train = X_train.view(X_train.size(0), -1).float() 
X_test = X_test.view(X_test.size(0), -1).float() 

In [4]:
# build custom module for logistic regression
class LogisticRegressionModel(torch.nn.Module):    
    # build the constructor
    def __init__(self, n_inputs, n_outputs):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(n_inputs, n_outputs)
    # make predictions
    # def forward(self, x):
    #     y_pred = torch.sigmoid(self.linear(x))
    #     return y_pred
    def forward(self, x):
        return self.linear(x)


# Model parameters
input_size = X_train.shape[1]  # no. of features (64 for digits dataset)
num_classes = 10  # (10 for digits dataset)

# Create the Logistic Regression model
model = LogisticRegressionModel(input_size, num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)


In [8]:
# Training function
def train_model(model, X_train, y_train, criterion, optimizer, num_epochs=100):
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()

        # Forward pass
        outputs = model(X_train)
        loss = criterion(outputs, y_train)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Print loss every 10 epochs
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Train the model for 100 epochs
train_model(model, X_train, y_train, criterion, optimizer, num_epochs=100)


Epoch [10/100], Loss: 1.6872
Epoch [20/100], Loss: 1.6825
Epoch [30/100], Loss: 1.6778
Epoch [40/100], Loss: 1.6740
Epoch [50/100], Loss: 1.6705
Epoch [60/100], Loss: 1.6671
Epoch [70/100], Loss: 1.6635
Epoch [80/100], Loss: 1.6601
Epoch [90/100], Loss: 1.6559
Epoch [100/100], Loss: 1.6503


In [13]:
import time

def evaluate_model(model, X_test, y_test):
    model.eval()
    start_time = time.time()

    with torch.no_grad():
        outputs = model(X_test)
        _, predicted = torch.max(outputs.data, 1)
        accuracy = (predicted == y_test).sum().item() / len(y_test)

    inference_time = time.time() - start_time
    model_size = sum(p.numel() * (4 if p.dtype == torch.float32 else 1) for p in model.parameters())

    return accuracy, model_size, inference_time

accuracy, model_size, inference_time = evaluate_model(model, X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%, Model Size: {model_size / 1e6:.2f} MB, Inference Time: {inference_time:.6f} seconds')


Accuracy: 82.24%, Model Size: 0.03 MB, Inference Time: 0.003699 seconds


In [15]:
from torch.quantization import quantize_dynamic

def quantize_model(model):
    torch.backends.quantized.engine = 'qnnpack'  # Set the quantization backend
    quantized_model = quantize_dynamic(
        model,  # The model to be quantized
        {torch.nn.Linear},  # Specify layers to be quantized (only Linear here)
        dtype=torch.qint8  # The target dtype for the quantized model
    )
    return quantized_model


In [18]:
import time

def evaluate_quantized_model(model, X_test, y_test):
    model.eval()
    start_time = time.time()
    X_test_tensor = X_test.view(X_test.size(0), -1).float()  # Flatten the test data
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs.data, 1)
    accuracy = (predicted == y_test).sum().item() / len(y_test)
    inference_time = time.time() - start_time
    model_size_bytes = sum(p.numel() * (4 if p.dtype == torch.float32 else 1) for p in model.parameters())
    model_size_mb = model_size_bytes / (1024 * 1024)  # Convert model size to MB
    
    return accuracy, model_size_mb, inference_time


In [20]:
quantized_model = quantize_model(model)  

accuracy_quant, model_size_quant, inference_time_quant = evaluate_quantized_model(quantized_model, X_test, y_test)
print(f'Quantized Accuracy: {accuracy_quant}, Model Size: {model_size_quant:.2f} MB, Quantized Inference Time: {inference_time_quant:.6f} seconds')

RuntimeError: quantized engine QNNPACK is not supported